In [1]:
import os

In [2]:
%pwd

'c:\\datascience\\Projects\\text_summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\datascience\\Projects\\text_summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import create_directory,read_yaml_config

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml_config(config_filepath)
        self.params = read_yaml_config(params_filepath)

        create_directory([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation
        create_directory([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

Logger module loaded successfully!


In [7]:
import os
from textSummarizer.logger_config.config import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


c:\datascience\Projects\text_summarizer\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-07 10:37:03,390: INFO: PyTorch version 2.6.0 available.]


In [8]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-04-07 10:37:05,692: INFO: Successfully loaded configuration from: config\config.yaml]
[2025-04-07 10:37:05,696: INFO: Successfully loaded configuration from: params.yaml]
[2025-04-07 10:37:05,698: INFO: Created directory at: artifacts]
[2025-04-07 10:37:05,699: INFO: Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\datascience\Projects\text_summarizer\env\lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 87685.05 examples/s] 
